In [1]:
from google.colab import drive

drive.mount('/content/gdrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = '/content/gdrive/My Drive/linkedin_a.txt'

In [0]:
text = open(path, errors = 'ignore').read()

In [4]:
print('corpus length:', len(text))

corpus length: 6889558


In [5]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM, Input
from keras.layers.merge import concatenate
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from keras.layers.normalization import BatchNormalization
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.decomposition import PCA
from keras.utils import plot_model
import numpy as np
import random
import sys
import csv
import os
import h5py
import time

embeddings_path =  "/content/gdrive/My Drive/glove.840B.300d-char.txt"
embedding_dim = 300
batch_size = 128
use_pca = False
lr = 0.001
lr_decay = 1e-4
maxlen = 10
consume_less = 2   # 0 for cpu, 2 for gpu

text = open(path, errors= 'ignore').read()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Using TensorFlow backend.


corpus length: 6889558
total chars: 95


In [6]:
# cut the text in semi-redundant sequences of maxlen characters

step = 6
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 1148258


In [7]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.int)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


Vectorization...


In [0]:
# test code to sample on 10% for functional model testing

def random_subset(X, y, p=0.1):

    idx = np.random.randint(X.shape[0], size=int(X.shape[0] * p))
    X = X[idx, :]
    y = y[idx]
    return (X, y)

In [9]:
print('Processing pretrained character embeds...')
embedding_vectors = {}
with open(embeddings_path, 'r') as f:
    for line in f:
        line_split = line.strip().split(" ")
        vec = np.array(line_split[1:], dtype=float)
        char = line_split[0]
        embedding_vectors[char] = vec


Processing pretrained character embeds...


In [0]:
embedding_matrix = np.zeros((len(chars), 300))
#embedding_matrix = np.random.uniform(-1, 1, (len(chars), 300))
for char, i in char_indices.items():
    #print ("{}, {}".format(char, i))
    embedding_vector = embedding_vectors.get(char)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [0]:
# Use PCA from sklearn to reduce 300D -> 50D
if use_pca:
    pca = PCA(n_components=embedding_dim)
    pca.fit(embedding_matrix)
    embedding_matrix_pca = np.array(pca.transform(embedding_matrix))
    print (embedding_matrix_pca)
    print (embedding_matrix_pca.shape)

In [10]:
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import RMSprop
model = Sequential()
model.add(LSTM(128, input_shape = (10, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               114688    
_________________________________________________________________
dense_2 (Dense)              (None, 95)                12255     
_________________________________________________________________
activation_2 (Activation)    (None, 95)                0         
Total params: 126,943
Trainable params: 126,943
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(X, y, batch_size = 128, epochs = 5)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/5





1148258/1148258 [==============================] - 194s 169us/step - loss: 2.8387 - acc: 0.1874
Epoch 2/5
1148258/1148258 [==============================] - 189s 164us/step - loss: 2.8120 - acc: 0.1960
Epoch 3/5
1148258/1148258 [==============================] - 189s 165us/step - loss: 2.8164 - acc: 0.1965
Epoch 4/5
1148258/1148258 [==============================] - 187s 163us/step - loss: 2.8219 - acc: 0.1965
Epoch 5/5
1148258/1148258 [==============================] - 186s 162us/step - loss: 2.8322 - acc: 0.1956
